In [55]:
# Statistical testing akin to that performed in the Saylor and Sundell paper

In [56]:
from __future__ import division, print_function, absolute_import
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy import stats
from scipy import signal
from scipy.signal import find_peaks
import random

In [57]:
def K(x, xi, h):
    return np.exp(-0.5*((x - xi)**2) / (h**2))* (1/(h*np.sqrt(2*np.pi)))

def kde_calc_plot(file_name, width, height, c, lower, upper, nums, num_bins, color_list, hist_input, kde_input, save=False,
                 single_data=False, single_dname=None, plot=True, return_y=False, similarity_test=False):
    # Creates a pandas data frame from an excel file and a specified column
    df = pd.read_excel(file_name)

    if single_data == True:
        new_list = [single_dname]
        color_list = [color_list[0]]
        error_list = [single_dname + ' Error']
    else:
        # Creates a list of the columns in the data frame
        column_list = df.columns.tolist()
        # List comprehension creating a list of the non-error columns in the data frame
        new_list = [column for column in column_list if 'Error' not in column]
        # List comprehension creating a list of the error columns in the data frame
        error_list = [column for column in column_list if 'Error' in column]

    # Provides length of new_list; used later on during plotting
    num_samples = len(new_list)
    counter = 0

    if plot == True:
        fig, ax = plt.subplots(len(new_list), 1, figsize=(width, height))
        plt.subplots_adjust(wspace = 0.5, hspace = 0.5)
    
    # Joins corresponding values in new_list, color_list, and error_list and iterates through them
    for column, the_color, error_col in zip(new_list, color_list, error_list):
        mask = ~np.isnan(df[column]) & ~np.isnan(df[error_col])
        # Eliminates all NaN entries in a specified non-error column of the data frame
        data = df[column][mask]
        # Eliminates all NaN entries in a specified error column of the data frame
        errors = df[error_col][mask]
        # Finds the maximum value of the error values in error_list
        error_max = max(errors)
        # Creates an array of evenly spaced numbers over a specified interval
        x_vals = np.linspace(lower, upper, nums)
        # Returns an all-zero array with the same shape and data type as the input
        total_sum = np.zeros_like(x_vals)
        # Sets up subplots and specifies their size
    
        # Joins corresponding xi and e values in data and errors and iterates through them
        for xi, e in zip(data, errors):
            # Sets the bandwidth (h) equal to the square root of error_max times by a c-value slightly greater than 1
            # squared minus e squared
            h = np.sqrt((c*error_max)**2 - (e)**2)
            # Runs x_vals - xi divided by h through the previously defined KDE function
            # Adds the result of this calculation to the array in total_sum
            total_sum += K(x_vals, xi, h)
        # Completes the KDE function by dividing total_sum by h times the number of data points in the column
        y_vals = total_sum/len(data)

        if plot == True:
            # Plots histogram and KDE
            if hist_input == True and kde_input == True:
                if single_data == True:
                    ax = [ax]
                ax[counter].plot(x_vals, y_vals, color='black')
                
                peaks, _ = find_peaks(y_vals, distance=20)
                ax[counter].plot(x_vals[peaks], y_vals[peaks], 'x', color='gray')
                peak_vals = x_vals[peaks].tolist()
                cell_vals = []

                for val in peak_vals:
                    new_val = round(val)
                    cell_vals.append(new_val)
                    
                for val1, val2, val3 in zip(cell_vals, x_vals[peaks], y_vals[peaks]):
                    ax[counter].annotate(f'{val1}', (val2, val3))
                    
                ax[counter].hist(df[column], color=the_color, edgecolor='white', bins=range(lower, upper, num_bins), density=True)
                ax[counter].text(0.05, 0.8, column + ' (n = ' + str(len(data)) + ')', transform=ax[counter].transAxes)
                
            # Plots only KDE
            elif kde_input == True:
                if single_data == True:
                    ax = [ax]
                ax[counter].plot(x_vals, y_vals, color=the_color)
                
                peaks, _ = find_peaks(y_vals, distance = 20)
                ax[counter].plot(x_vals[peaks], y_vals[peaks], 'x', color='gray')
                peak_vals = x_vals[peaks].tolist()
                cell_vals = []
                
                for val in peak_vals:
                    new_val = round(val)
                    cell_vals.append(new_val)
                    
                for val1, val2, val3 in zip(cell_vals, x_vals[peaks], y_vals[peaks]):
                    ax[counter].annotate(f'{val1}', (val2, val3))
                
                ax[counter].text(0.05, 0.8, column + ' (n = ' + str(len(data)) + ')', transform=ax[counter].transAxes)
            
            # Plots only histogram
            elif hist_input == True:
                if single_data == True:
                    ax = [ax]
                ax.hist(df[column], color=the_color, edgecolor='white', bins=range(lower, upper, num_bins), density=True)
                ax.text(0.05, 0.8, column + ' (n = ' + str(len(data)) + ')', transform=ax[counter].transAxes)
            
            counter += 1
            
    # Labels x-axis
    if plot == True:
        plt.xlabel('Age (Ma)')
        
    # Saves final graphs as pngs
    if save == True:
        print('working')
        plt.savefig('c-'+str(c).split('.')[1]+'.png', dpi=400, bbox_inches='tight')

    # Shows final graphs
    plt.show()

    if return_y == True:
        return y_vals

    elif (return_y == True) & (similarity_test == True):
        return total_sum/sum(total_sum)

In [77]:
df = pd.read_excel('Potsdam-StatTest.xlsx')
column_list = df.columns.tolist()
new_list = [column for column in column_list if 'Error' not in column]
error_list = [column for column in column_list if 'Error' in column]

df1 = pd.read_excel('FL3-StatTest.xlsx')
column_list1 = df1.columns.tolist()
new_list1 = [column for column in column_list if 'Error' not in column]
error_list1 = [column for column in column_list if 'Error' in column]

popped_data = new_list1.pop()
new_list.append(popped_data)
popped_error = error_list1.pop()
error_list.append(popped_error)
general_list = []
    
# Joins corresponding values in new_list, color_list, and error_list and iterates through them
for column, error_col in zip(new_list, error_list):
    mask = ~np.isnan(df[column]) & ~np.isnan(df[error_col])
    # Eliminates all NaN entries in a specified non-error column of the data frame
    data = df[column][mask].tolist()
    # Eliminates all NaN entries in a specified error column of the data frame
    errors = df[error_col][mask].tolist()
    zipped_set = list(zip(data, errors))
    random_set = random.sample(zipped_set, 25)
    new_errors = [couple[1] for couple in random_set]
    # Finds the maximum value of the error values in new_errors
    error_max = max(new_errors)
    # Creates an array of evenly spaced numbers over a specified interval
    x_vals = np.linspace(0, 3000, 1000)
    # Returns an all-zero array with the same shape and data type as the input
    total_sum = np.zeros_like(x_vals)
    # Sets up subplots and specifies their size
    
    # Joins corresponding xi and e values in data and errors and iterates through them
    for xi, e in random_set:
        # Sets the bandwidth (h) equal to the square root of error_max times by a c-value slightly greater than 1
        # squared minus e squared
        h = np.sqrt((1.04*error_max)**2 - (e)**2)
        # Runs x_vals - xi divided by h through the previously defined KDE function
        # Adds the result of this calculation to the array in total_sum
        total_sum += K(x_vals, xi, h)
    # Completes the KDE function by dividing total_sum by h times the number of data points in the column
    y_vals = total_sum/len(data)
    general_list.append(y_vals)
    break

print(y_vals)

[3.19613890e-39 5.22474974e-39 8.52776213e-39 1.38974339e-38
 2.26133201e-38 3.67387562e-38 5.95957386e-38 9.65243400e-38
 1.56095218e-37 2.52042471e-37 4.06339921e-37 6.54089456e-37
 1.05127653e-36 1.68705493e-36 2.70317521e-36 4.32466204e-36
 6.90817687e-36 1.10181408e-35 1.75463620e-35 2.78997339e-35
 4.42942713e-35 7.02150012e-35 1.11134123e-34 1.75630709e-34
 2.77133690e-34 4.36630817e-34 6.86872267e-34 1.07888330e-33
 1.69203840e-33 2.64961989e-33 4.14280911e-33 6.46762203e-33
 1.00816840e-32 1.56913625e-32 2.43852796e-32 3.78385711e-32
 5.86248866e-32 9.06922131e-32 1.40087403e-31 2.16057663e-31
 3.32722503e-31 5.11607615e-31 7.85478966e-31 1.20413525e-30
 1.84314518e-30 2.81700620e-30 4.29893085e-30 6.55055253e-30
 9.96645349e-30 1.51408064e-29 2.29669561e-29 3.47859892e-29
 5.26080463e-29 7.94414749e-29 1.19781609e-28 1.80335450e-28
 2.71094433e-28 4.06919980e-28 6.09883303e-28 9.12712872e-28
 1.36386596e-27 2.03497776e-27 3.03178550e-27 4.51012380e-27
 6.69930882e-27 9.936270

In [61]:
df = pd.read_excel('FL3-StatTest.xlsx')
column_list = df.columns.tolist()
# List comprehension creating a list of the non-error columns in the data frame
new_list = [column for column in column_list if 'Error' not in column]
color_list = ['black']
# List comprehension creating a list of the error columns in the data frame
error_list = [column for column in column_list if 'Error' in column]
# Provides length of new_list; used later on during plotting
num_samples = len(new_list)
counter = 0
    
# Joins corresponding values in new_list, color_list, and error_list and iterates through them
for column, the_color, error_col in zip(new_list, color_list, error_list):
    mask = ~np.isnan(df[column]) & ~np.isnan(df[error_col])
    # Eliminates all NaN entries in a specified non-error column of the data frame
    data = df[column][mask].tolist()
    # Eliminates all NaN entries in a specified error column of the data frame
    errors = df[error_col][mask].tolist()
    zipped_set = list(zip(data, errors))
    random_set = random.sample(zipped_set, 25)
    new_errors = [couple[1] for couple in random_set]
    # Finds the maximum value of the error values in new_errors
    error_max = max(new_errors)
    # Creates an array of evenly spaced numbers over a specified interval
    x_vals = np.linspace(0, 3000, 1000)
    # Returns an all-zero array with the same shape and data type as the input
    total_sum = np.zeros_like(x_vals)
    # Sets up subplots and specifies their size
    
    # Joins corresponding xi and e values in data and errors and iterates through them
    for xi, e in random_set:
        # Sets the bandwidth (h) equal to the square root of error_max times by a c-value slightly greater than 1
        # squared minus e squared
        h = np.sqrt((1.04*error_max)**2 - (e)**2)
        # Runs x_vals - xi divided by h through the previously defined KDE function
        # Adds the result of this calculation to the array in total_sum
        total_sum += K(x_vals, xi, h)
    # Completes the KDE function by dividing total_sum by h times the number of data points in the column
    y_vals1 = total_sum/len(data)
    break
print(y_vals1)

[2.72124549e-193 6.67269100e-192 1.61694538e-190 3.87213716e-189
 9.16362185e-188 2.14311083e-186 4.95316899e-185 1.13131281e-183
 2.55354380e-182 5.69593416e-181 1.25558944e-179 2.73521452e-178
 5.88838472e-177 1.25274306e-175 2.63383690e-174 5.47238690e-173
 1.12363629e-171 2.28000481e-170 4.57200624e-169 9.06022349e-168
 1.77432042e-166 3.43388887e-165 6.56752094e-164 1.24130282e-162
 2.31854249e-161 4.27970095e-160 7.80679601e-159 1.40732138e-157
 2.50711804e-156 4.41384750e-155 7.67928722e-154 1.32033928e-152
 2.24342326e-151 3.76702012e-150 6.25094539e-149 1.02507234e-147
 1.66120936e-146 2.66045103e-145 4.21063129e-144 6.58567359e-143
 1.01792138e-141 1.55485292e-140 2.34706672e-139 3.50124581e-138
 5.16155803e-137 7.51969231e-136 1.08263078e-134 1.54035817e-133
 2.16582872e-132 3.00945284e-131 4.13249201e-130 5.60786495e-129
 7.52045497e-128 9.96670786e-127 1.30533020e-125 1.68946846e-124
 2.16093069e-123 2.73144606e-122 3.41197206e-121 4.21191256e-120
 5.13823867e-119 6.194556

In [62]:
# Hypothesis test for likeness coefficient, n = 25
new_list = [y_vals, y_vals1]
new_tuple = tuple(new_list)
M = np.sum(np.abs(new_tuple[0] - new_tuple[1]))/2
L = 1 - M
print(L)

0.9827902618179778


In [74]:
df = pd.read_excel('Potsdam-StatTest.xlsx')
column_list = df.columns.tolist()
# List comprehension creating a list of the non-error columns in the data frame
new_list = [column for column in column_list if 'Error' not in column]
color_list = ['black']
# List comprehension creating a list of the error columns in the data frame
error_list = [column for column in column_list if 'Error' in column]
# Provides length of new_list; used later on during plotting
num_samples = len(new_list)
counter = 0
    
# Joins corresponding values in new_list, color_list, and error_list and iterates through them
for column, the_color, error_col in zip(new_list, color_list, error_list):
    mask = ~np.isnan(df[column]) & ~np.isnan(df[error_col])
    # Eliminates all NaN entries in a specified non-error column of the data frame
    data = df[column][mask].tolist()
    # Eliminates all NaN entries in a specified error column of the data frame
    errors = df[error_col][mask].tolist()
    zipped_set = list(zip(data, errors))
    random_set = random.sample(zipped_set, 100)
    new_errors = [couple[1] for couple in random_set]
    # Finds the maximum value of the error values in new_errors
    error_max = max(new_errors)
    # Creates an array of evenly spaced numbers over a specified interval
    x_vals = np.linspace(0, 3000, 1000)
    # Returns an all-zero array with the same shape and data type as the input
    total_sum = np.zeros_like(x_vals)
    # Sets up subplots and specifies their size
    
    # Joins corresponding xi and e values in data and errors and iterates through them
    for xi, e in random_set:
        # Sets the bandwidth (h) equal to the square root of error_max times by a c-value slightly greater than 1
        # squared minus e squared
        h = np.sqrt((1.04*error_max)**2 - (e)**2)
        # Runs x_vals - xi divided by h through the previously defined KDE function
        # Adds the result of this calculation to the array in total_sum
        total_sum += K(x_vals, xi, h)
    # Completes the KDE function by dividing total_sum by h times the number of data points in the column
    y_vals2 = total_sum/len(data)
    break
print(y_vals2)

[2.13563061e-18 2.68866415e-18 3.38186051e-18 4.24994757e-18
 5.33605425e-18 6.69369187e-18 8.38919035e-18 1.05046907e-17
 1.31418169e-17 1.64261777e-17 2.05128793e-17 2.55932697e-17
 3.19031821e-17 3.97329999e-17 4.94399333e-17 6.14629790e-17
 7.63411312e-17 9.47355273e-17 1.17456353e-16 1.45495493e-16
 1.80066125e-16 2.22650662e-16 2.75058781e-16 3.39497544e-16
 4.18656094e-16 5.15807805e-16 6.34933397e-16 7.80869142e-16
 9.59485110e-16 1.17789931e-15 1.44473471e-15 1.77042742e-15
 2.16759574e-15 2.65148187e-15 3.24047979e-15 3.95676561e-15
 4.82704954e-15 5.88347196e-15 7.16467008e-15 8.71704645e-15
 1.05962759e-14 1.28690941e-14 1.56154173e-14 1.89308538e-14
 2.29296743e-14 2.77483228e-14 3.35495617e-14 4.05273605e-14
 4.89126555e-14 5.89801288e-14 7.10561786e-14 8.55282801e-14
 1.02855969e-13 1.23583712e-13 1.48355982e-13 1.77934883e-13
 2.13220744e-13 2.55276161e-13 3.05354014e-13 3.64930114e-13
 4.35741175e-13 5.19828944e-13 6.19591443e-13 7.37842390e-13
 8.77880036e-13 1.043566

In [75]:
df = pd.read_excel('FL3-StatTest.xlsx')
column_list = df.columns.tolist()
# List comprehension creating a list of the non-error columns in the data frame
new_list = [column for column in column_list if 'Error' not in column]
color_list = ['black']
# List comprehension creating a list of the error columns in the data frame
error_list = [column for column in column_list if 'Error' in column]
# Provides length of new_list; used later on during plotting
num_samples = len(new_list)
counter = 0
    
# Joins corresponding values in new_list, color_list, and error_list and iterates through them
for column, the_color, error_col in zip(new_list, color_list, error_list):
    mask = ~np.isnan(df[column]) & ~np.isnan(df[error_col])
    # Eliminates all NaN entries in a specified non-error column of the data frame
    data = df[column][mask].tolist()
    # Eliminates all NaN entries in a specified error column of the data frame
    errors = df[error_col][mask].tolist()
    zipped_set = list(zip(data, errors))
    random_set = random.sample(zipped_set, 100)
    new_errors = [couple[1] for couple in random_set]
    # Finds the maximum value of the error values in new_errors
    error_max = max(new_errors)
    # Creates an array of evenly spaced numbers over a specified interval
    x_vals = np.linspace(0, 3000, 1000)
    # Returns an all-zero array with the same shape and data type as the input
    total_sum = np.zeros_like(x_vals)
    # Sets up subplots and specifies their size
    
    # Joins corresponding xi and e values in data and errors and iterates through them
    for xi, e in random_set:
        # Sets the bandwidth (h) equal to the square root of error_max times by a c-value slightly greater than 1
        # squared minus e squared
        h = np.sqrt((1.04*error_max)**2 - (e)**2)
        # Runs x_vals - xi divided by h through the previously defined KDE function
        # Adds the result of this calculation to the array in total_sum
        total_sum += K(x_vals, xi, h)
    # Completes the KDE function by dividing total_sum by h times the number of data points in the column
    y_vals3 = total_sum/len(data)
    break
print(y_vals3)

[9.93749863e-166 2.32854371e-164 5.38319726e-163 1.22784849e-161
 2.76310793e-160 6.13478725e-159 1.34384676e-157 2.90434749e-156
 6.19292740e-155 1.30284271e-153 2.70418608e-152 5.53770406e-151
 1.11884905e-149 2.23029259e-148 4.38632460e-147 8.51115072e-146
 1.62938750e-144 3.07757797e-143 5.73511769e-142 1.05444563e-140
 1.91273424e-139 3.42321092e-138 6.04451239e-137 1.05302206e-135
 1.80993199e-134 3.06927375e-133 5.13520258e-132 8.47672554e-131
 1.38053445e-129 2.21827284e-128 3.51666711e-127 5.50042369e-126
 8.48808293e-125 1.29232477e-123 1.94125376e-122 2.87701110e-121
 4.20677564e-120 6.06884085e-119 8.63795092e-118 1.21300978e-116
 1.68060812e-115 2.29729728e-114 3.09825047e-113 4.12253581e-112
 5.41203922e-111 7.00980632e-110 8.95776548e-109 1.12938481e-107
 1.40485915e-106 1.72413874e-105 2.08766218e-104 2.49400213e-103
 2.93955780e-102 3.41834407e-101 3.92191456e-100 4.43944898e-099
 4.95802366e-098 5.46306927e-097 5.93900109e-096 6.36998909e-095
 6.74081731e-094 7.037769

In [76]:
new_list = [y_vals2, y_vals3]
new_tuple = tuple(new_list)
M = np.sum(np.abs(new_tuple[0] - new_tuple[1]))/2
L = 1 - M
print(L)

0.9301230510669718
